In [1]:
import pysolr

solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url, always_commit=True)

In [2]:
from backend.lql.antlr_parser import parse_interface_spec

lql_string = """
    Calculator {
        Calculator(int)->None
        mean(list)->float
        sum(list)->float
    }
    """

interfaceSpecification = parse_interface_spec(lql_string)

Interface Specification: InterfaceSpecification(className=Calculator, constructors=[MethodSignature(methodName=Calculator, returnType=None, parameterTypes=['int'])], methods=[MethodSignature(methodName=mean, returnType=float, parameterTypes=['list']), MethodSignature(methodName=sum, returnType=float, parameterTypes=['list'])])


In [3]:
from backend.arena import solr_query
import importlib
importlib.reload(solr_query)

solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url)
solr_queries = solr_query.translate_to_solr_queries(interfaceSpecification)
solr_queries

['(methodSignatureParamsOrderedNodefault:1*) OR (methodSignatureParamsOrderedNodefault:*pt_list*) OR ((method:mean) AND (methodSignatureParamsOrderedNodefault:1*))^2 OR ((method:mean) AND (methodSignatureParamsOrderedNodefault:*pt_list*))^2 OR ((methodSignatureParamsOrderedNodefault:*pt_list*) OR (methodSignatureParamsOrderedNodefault:1*))^3',
 '(methodSignatureParamsOrderedNodefault:1*) OR (methodSignatureParamsOrderedNodefault:*pt_list*) OR ((method:sum) AND (methodSignatureParamsOrderedNodefault:1*))^2 OR ((method:sum) AND (methodSignatureParamsOrderedNodefault:*pt_list*))^2 OR ((methodSignatureParamsOrderedNodefault:*pt_list*) OR (methodSignatureParamsOrderedNodefault:1*))^3']

In [4]:
results = []
for query in solr_queries:
    results += solr.search(query)

In [5]:
from backend.arena import solr_parser
from backend.arena.solr_parser import parse_solr_response
importlib.reload(solr_parser)
allModulesUnderTest = parse_solr_response(results)
moduleUnderTest = allModulesUnderTest[0]

numpy._core.fromnumeric
numpy._core.arrayprint
Generated 2 ModuleUnderTest objects.
